# **100 Cities Demographic Analysis** 

## **Imports and Data**

In [17]:
import os, sys, requests, zipfile, pathlib
from pathlib import Path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# Vineyard functions
from VineyardFinal import vineyard_from_pds, Get_Adjacency_Persistence, Homotopy, Get_W_Infinity, vdist, min_vc, fD, fL

# Matrix Operations
import numpy as np
import pandas as pd

# misc
from tqdm import tqdm

# tda
import gudhi
import gudhi.hera

# Geospatial and graphs
import geopandas as gpd
from gerrychain import Graph
import networkx as nx

import warnings
warnings.filterwarnings('ignore', message='Found islands', category=UserWarning) # Getting rid of pesky degree-0 graph warnings 

INFINITY = 1e6
n = 100

## **Data** 

In [18]:
# download helpers #
def download(url, out_file):

    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with out_file.open("wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)

    print(f"Downloaded: {out_file}.")

def download_zip(url, out_zip, extract_to):

    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with out_zip.open("wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
    
    with zipfile.ZipFile(out_zip) as zf:
        zf.extractall(extract_to)
    
    ## cleanup ##
    if os.exists(out_zip): os.remove(out_zip) 

    print(f"Downloaded: {out_zip.resolve()}")
    print(f"Extracted to: {extract_to.resolve()}")

###### City Data ######
# src: https://github.com/thomasweighill/tda-cities/blob/main/cities2020data.zip
# src: https://github.com/thomasweighill/tda-cities/blob/main/City_Names_And_Populations.csv

# os.makedirs("../dataIn/cities/", exist_ok=True)
# download_zip('https://github.com/thomasweighill/tda-cities/raw/main/cities2020data.zip', Path("../dataIn/cities/cities2020data.zip"), Path("../dataIn/cities/")) 
# download('https://github.com/thomasweighill/tda-cities/raw/main/City_Names_And_Populations.csv', Path("../dataIn/cities/City_Names_And_Populations.csv"))

## **Metric Calculation**

In [19]:
###### Analysis Params ###### 

# Demographic Cols #
col1 = 'BLACK'
col2 = 'HISP'

# length of homotopy #
t = 100

# cities
list_of_cities_pd =pd.read_csv('../dataIn/cities/City_Names_And_Populations.csv')
city_names = [x + y for x,y in zip(list_of_cities_pd.NAME, list_of_cities_pd.ST)][:n]

###### Out Dir ###### 

outDir = '../dataOut/'
os.makedirs(outDir, exist_ok = True)


cols = [
    'w',
    'w1',
    'v',
    'mvc',
    'v/mvc',
    'l1',
    'l1w',
    'l2',
    'l2w',
    'linf',
    'linfw',
    'R',
    'R^2',
    f'{col1}%',
    f'{col2}%',
    'TotPop',
]

df_out = pd.DataFrame(columns = cols)

for city in tqdm(city_names, desc = "Processing Cities"):
    # read graph
    G = Graph.from_json(f"../dataIn/cities/cities2020data/{city}.json") # cities2020data
    # Tract population
    pop = np.array(list((nx.get_node_attributes(G, 'TOTPOP').values())))

    # f and g are both percentages of some statistic over a population
    f = np.array(list((nx.get_node_attributes(G,col1).values()))) / pop
    g = np.array(list((nx.get_node_attributes(G,col2).values()))) / pop

    # Fill NAN 
    f = np.nan_to_num(f, 0)
    g = np.nan_to_num(g, 0)


###### Compute Vineyard ###### 

    # Compute persistence diagrams
    PD0 = []

    # Straight line homotopy
    Hs = Homotopy(f,g,t = t)
    for h in Hs:
        for i, v in enumerate(G.nodes()):
            G.nodes[v]['Homotopy'] = h[i]
            
        PD0.append(Get_Adjacency_Persistence(G,'Homotopy', dimension = 0, popCol = "TOTPOP", popMin = 10))

###### Calculate Metrics ###### 

    # Wasserstein Distance
    w = Get_W_Infinity(PD0[0], PD0[-1])
    # 1-Wasserstein Distance
    w1 = gudhi.hera.wasserstein_distance(PD0[0], PD0[-1])
    # Vineyard Distance (F(D) = D, F(L) = 1)
    v = vdist(vineyard_from_pds(PD0), fD, fL)
    # MVC 
    mvc = min_vc(PD0[0], PD0[-1])
    # v/mvc
    vmvc = v / mvc
    #l1 Norm
    l1 = np.linalg.norm(f - g, ord = 1)
    # l2 Norm
    l2 = np.linalg.norm(f - g, ord = 2)
    #Linf Norm
    linf = np.linalg.norm(f - g, ord = np.inf)
    # Weighted l1 Norm
    l1w = l1 / len(G)
    # Weighted L2 Norm
    l2w = l2 / len(G)
    # Weighted Linf Norm
    linfw = linf / len(G)
    # Pearson Correlation
    r = np.corrcoef(f,g)[0,1]
    # R^2
    r2 = r ** 2
    # Total Population
    totPop = np.sum(pop)
    # col1%
    col1P = np.sum(f * pop) / totPop * 100.0
    # col2%
    col2P = np.sum(g * pop) / totPop * 100.0

###### Store Data ###### 
    data = pd.Series([w, w1, v, mvc, vmvc, l1, l1w, l2, l2w, linf, linfw, r, r2, col1P, col2P, totPop])

    df_out.loc[city] = pd.Series({
        f'{col}' : d for col, d in zip(cols, data)
    })

    print(f'{city} done')
    
###### Save ###### 
df_out.index.name = 'Name'
df_out.to_csv(os.path.join(outDir, 'city_metrics.csv'), index = True)
print(f"Metric calculation complete. \nResults saved to {outDir}/city_metrics.csv")

Processing Cities:   0%|          | 0/100 [00:00<?, ?it/s]/var/folders/_b/bl5yw1q15msg5ky9z_04dpt40000gn/T/ipykernel_97676/3134900562.py:48: RuntimeWarning: invalid value encountered in divide
  f = np.array(list((nx.get_node_attributes(G,col1).values()))) / pop
Processing Cities:   1%|          | 1/100 [00:07<11:38,  7.06s/it]

New YorkNY done


/var/folders/_b/bl5yw1q15msg5ky9z_04dpt40000gn/T/ipykernel_97676/3134900562.py:48: RuntimeWarning: invalid value encountered in divide
  f = np.array(list((nx.get_node_attributes(G,col1).values()))) / pop
Processing Cities:   2%|▏         | 2/100 [00:10<07:34,  4.64s/it]

Los AngelesCA done


/var/folders/_b/bl5yw1q15msg5ky9z_04dpt40000gn/T/ipykernel_97676/3134900562.py:48: RuntimeWarning: invalid value encountered in divide
  f = np.array(list((nx.get_node_attributes(G,col1).values()))) / pop
Processing Cities:   3%|▎         | 3/100 [00:11<04:57,  3.06s/it]

...

North Las VegasNV done
IrvingTX done
FremontCA done


Processing Cities:  96%|█████████▌| 96/100 [00:27<00:00, 13.74it/s]/var/folders/_b/bl5yw1q15msg5ky9z_04dpt40000gn/T/ipykernel_97676/3134900562.py:48: RuntimeWarning: invalid value encountered in divide
  f = np.array(list((nx.get_node_attributes(G,col1).values()))) / pop
Processing Cities:  98%|█████████▊| 98/100 [00:27<00:00, 13.35it/s]

IrvineCA done
BirminghamAL done
RochesterNY done


Processing Cities: 100%|██████████| 100/100 [00:28<00:00,  3.56it/s]

San BernardinoCA done
SpokaneWA done
Metric calculation complete. 
Results saved to ../dataOut//city_metrics.csv


In [20]:
df_1 = pd.read_csv('../../PaperCodes/DataOut/City_Metrics_P.csv', index_col = "Name")
df_2 = pd.read_csv('../dataOut/city_metrics.csv', index_col = "Name")
print(df_1['v'].head())
print(df_2['v'].head())
print(df_1.loc["DallasTX"]['v'])
print(df_2.loc["DallasTX"]['v'])

print(df_1.loc["DallasTX"]['w1'])
print(df_2.loc["DallasTX"]['w1'])

Name
New YorkNY        3.162003
Los AngelesCA     2.358659
ChicagoIL         1.479105
HoustonTX         2.211081
PhiladelphiaPA    0.835309
Name: v, dtype: float64
Name
New YorkNY        5.064414
Los AngelesCA     3.962156
ChicagoIL         2.433321
HoustonTX         3.644369
PhiladelphiaPA    1.293566
Name: v, dtype: float64
1.4905361117503764
2.2932280789814903
3.843986792849618
3.843986792849618
